In [2]:
import graphlab as gl

A newer version of GraphLab Create (v1.7.1) is available! Your current version is v1.6.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [3]:
data = gl.SFrame('people_wiki.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to kell.trollhammaren@hotmail.com and will expire on October 08, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-31170 - Server binary: /usr/local/lib/python2.7/dist-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1448246416.log
[INFO] GraphLab Server Version: 1.6.1


### Examine a particular famous person, 'Elton John'.
* 1 - What are the 3 words in his articles with highest word counts?
* 2 - What are the 3 words in his articles with highest TF-IDF?
* 3 - What’s the cosine distance between the articles on ‘Elton John’ and ‘Victoria Beckham’?
* 4 - What’s the cosine distance between the articles on ‘Elton John’ and Paul McCartney’?
* 5 - Which one of the two is closest to Elton John?
### Using Nearest Neghbors with Cosine XD

* 6 - What’s the most similar article, other than itself, to the one on ‘Elton John’ using word count features?
* 7 - What’s the most similar article, other than itself, to the one on ‘Elton John’ using TF-IDF features?
* 8 - What’s the most similar article, other than itself, to the one on ‘Victoria Beckham’ using word count features?
* 9 - What’s the most similar article, other than itself, to the one on ‘Victoria Beckham’ using TF-IDF features?

In [5]:
data['word_count'] = gl.text_analytics.count_words(data['text'])

In [6]:
data.head(2)

URI,name,text,word_count
<http://dbpedia.org/resource/Digby_Morrell> ...,Digby Morrell,digby morrell born 10october 1979 is a former ...,"{'since': 1, 'carltons':1, 'being': 1, '2005' ..."
<http://dbpedia.org/resource/Alfred_J._Lewy> ...,Alfred J. Lewy,alfred j lewy aka sandylewy graduated from ...,"{'precise': 1, 'thomas':1, 'closely': 1, ..."


In [10]:
tfidf = gl.text_analytics.tf_idf(data['word_count'])

In [11]:
tfidf.head(2)

docs
"{'since':1.455376717308041, ..."
"{'precise':6.44320060695519, ..."


In [12]:
data['tfidf'] = tfidf['docs']

In [13]:
data.head(2)

Columns:
	URI	str
	name	str
	text	str
	word_count	dict
	tfidf	dict

Rows: 2

Data:
+-------------------------------+----------------+
|              URI              |      name      |
+-------------------------------+----------------+
| <http://dbpedia.org/resour... | Digby Morrell  |
| <http://dbpedia.org/resour... | Alfred J. Lewy |
+-------------------------------+----------------+
+-------------------------------+-------------------------------+
|              text             |           word_count          |
+-------------------------------+-------------------------------+
| digby morrell born 10 octo... | {'since': 1, 'carltons': 1... |
| alfred j lewy aka sandy le... | {'precise': 1, 'thomas': 1... |
+-------------------------------+-------------------------------+
+-------------------------------+
|             tfidf             |
+-------------------------------+
| {'since': 1.45537671730804... |
| {'precise': 6.443200606955... |
+-------------------------------+
[2 rows x 5 columns]

In [14]:
john = data[data['name'] == 'Elton John']

In [16]:
john[['word_count']].stack('word_count', new_column_name = ['word', 'count']).sort('count', ascending=False)

word,count
the,27
in,18
and,15
of,13
a,10
has,9
he,7
john,7
on,6
since,5


In [17]:
john[['tfidf']].stack('tfidf', new_column_name = ['word', 'tfidf']).sort('tfidf', ascending=False)

word,tfidf
furnish,18.38947184
elton,17.48232027
billboard,17.3036809575
john,13.9393127924
songwriters,11.250406447
overallelton,10.9864953892
tonightcandle,10.9864953892
19702000,10.2933482087
fivedecade,10.2933482087
aids,10.262846934


In [23]:
victoria = data[data['name'] == 'Victoria Beckham']
paul = data[data['name'] == 'Paul McCartney']

In [24]:
gl.distances.cosine(john['tfidf'][0],victoria['tfidf'][0])

0.9567006376655429

In [25]:
gl.distances.cosine(john['tfidf'][0],paul['tfidf'][0])

0.8250310029221779

In [27]:
knn_model_tfidf = gl.nearest_neighbors.create(data, features=['tfidf'], label='name', distance='cosine')

PROGRESS: Starting brute force nearest neighbors model training.


In [29]:
knn_model_word_count = gl.nearest_neighbors.create(data, features=['word_count'], label='name', distance='cosine')

PROGRESS: Starting brute force nearest neighbors model training.


In [30]:
knn_model_tfidf.query(john)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.00169288  | 11.517ms     |
PROGRESS: | Done         |         | 100         | 300.023ms    |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,Elton John,-2.22044604925e-16,1
0,Rod Stewart,0.717219667893,2
0,George Michael,0.747600998969,3
0,Sting (musician),0.747671954431,4
0,Phil Collins,0.75119324879,5


In [31]:
knn_model_word_count.query(john)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.00169288  | 6.056ms      |
PROGRESS: | Done         |         | 100         | 278.636ms    |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,Elton John,2.22044604925e-16,1
0,Cliff Richard,0.16142415259,2
0,Sandro Petrone,0.16822542751,3
0,Rod Stewart,0.168327165587,4
0,Malachi O'Doherty,0.177315545979,5


In [32]:
knn_model_tfidf.query(victoria)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.00169288  | 10.343ms     |
PROGRESS: | Done         |         | 100         | 280.227ms    |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,Victoria Beckham,1.11022302463e-16,1
0,David Beckham,0.548169610263,2
0,Stephen Dow Beckham,0.784986706828,3
0,Mel B,0.809585523409,4
0,Caroline Rush,0.819826422919,5


In [33]:
knn_model_word_count.query(victoria)

PROGRESS: Starting pairwise querying.
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 1       | 0.00169288  | 8.19ms       |
PROGRESS: | Done         |         | 100         | 295.768ms    |
PROGRESS: +--------------+---------+-------------+--------------+


query_label,reference_label,distance,rank
0,Victoria Beckham,-2.22044604925e-16,1
0,Mary Fitzgerald (artist),0.207307036115,2
0,Adrienne Corri,0.214509782788,3
0,Beverly Jane Fry,0.217466468741,4
0,Raman Mundair,0.217695474992,5


# Answers
* 1 - the, in, and
* 2 - furnish, elton, billboard
* 3 - 0.9567006376655429
* 4 - 0.8250310029221779
* 5 - Paul McCartney XD
* 6 - Cliff Richard	
* 7 - Rod Stewart
* 8 - Mary Fitzgerald (artist)	
* 9 - David Beckham